## Introduction

In this project, i will divide Cleveland restaurants by calling the Foursquare API. The clusters will be visualized using Folium library.

### Import necessary Libraries

In [1]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/ibrahimseven/anaconda3

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          82 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge


geopy-1.17.0         | 49 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##



### Define Foursquare Credentials and Version

In [24]:
CLIENT_ID = 'XKNOVVX4SCW5XB4UYVKCE0UATV1XCLISRHRWPHQ0BTXHHBQO' # your Foursquare ID
CLIENT_SECRET = '3HGEKWCIKUVL3OFGU4BRHMNHPK5QQNNAUVMIYKPIFJVASWOX' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 300000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XKNOVVX4SCW5XB4UYVKCE0UATV1XCLISRHRWPHQ0BTXHHBQO
CLIENT_SECRET:3HGEKWCIKUVL3OFGU4BRHMNHPK5QQNNAUVMIYKPIFJVASWOX


#### We wanna convert public square Cleveland (dtown 0 POINT)  address to its latitude and longitude coordinates.

In [25]:
address = '230 W Huron Rd, Cleveland, OH'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

/home/ibrahimseven/anaconda3/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


41.4969221 -81.6934307


In [26]:
search_query = 'Restaurant'
radius = 50000
print(search_query + ' .... OK!')

Restaurant .... OK!


#### Send the GET Request and examine the results

In [27]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=XKNOVVX4SCW5XB4UYVKCE0UATV1XCLISRHRWPHQ0BTXHHBQO&client_secret=3HGEKWCIKUVL3OFGU4BRHMNHPK5QQNNAUVMIYKPIFJVASWOX&ll=41.4969221,-81.6934307&v=20180604&query=Restaurant&radius=50000&limit=300000'

In [28]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5bf9acb6351e3d163db41809'},
 'response': {'venues': [{'id': '4ad4bff0f964a52021e920e3',
    'name': 'Pickwick & Frolic Restaurant and Club',
    'location': {'address': '2035 E 4th St',
     'lat': 41.49914493913805,
     'lng': -81.69017656212975,
     'labeledLatLngs': [{'label': 'display',
       'lat': 41.49914493913805,
       'lng': -81.69017656212975}],
     'distance': 367,
     'postalCode': '44115',
     'cc': 'US',
     'city': 'Cleveland',
     'state': 'OH',
     'country': 'United States',
     'formattedAddress': ['2035 E 4th St',
      'Cleveland, OH 44115',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d14e941735',
      'name': 'American Restaurant',
      'pluralName': 'American Restaurants',
      'shortName': 'American',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'primary': True}],
    'venuePage': {'id': '52556740'},
    'referralId': 'v-1

#### Get relevant part of JSON and transform it into a *pandas* dataframe

In [31]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",NaN,NaN,NaN,NaN,NaN,NaN,False,4ad4bff0f964a52021e920e3,2035 E 4th St,...,367,"[2035 E 4th St, Cleveland, OH 44115, United St...","[{'label': 'display', 'lat': 41.49914493913805...",41.499145,-81.690177,44115,OH,Pickwick & Frolic Restaurant and Club,v-1543089334,52556740
1,"[{'id': '4bf58dd8d48988d14a941735', 'name': 'V...",NaN,NaN,NaN,NaN,NaN,NaN,False,4ae1cb35f964a520738721e3,2061 E 4th St,...,349,"[2061 E 4th St, Cleveland, OH 44115, United St...","[{'label': 'display', 'lat': 41.498871, 'lng':...",41.498871,-81.690149,44115,OH,Saigon Restaurant & Bar,v-1543089334,NaN
2,"[{'id': '4bf58dd8d48988d146941735', 'name': 'D...",283564,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/barristers-...,False,4cd6d9b3aeb1224b377f2aff,1424 Hamilton Ave,...,1234,"[1424 Hamilton Ave, Cleveland, OH 44114, Unite...","[{'label': 'display', 'lat': 41.50674797333065...",41.506748,-81.686566,44114,OH,Barrister's Deli- Restaurant- Catering,v-1543089334,418559616
3,"[{'id': '4bf58dd8d48988d157941735', 'name': 'N...",NaN,NaN,NaN,NaN,NaN,NaN,False,5748cb02498e1c0c6cfa6584,Hilton Cleveland Downtown,...,706,"[Hilton Cleveland Downtown, Cleveland, OH 4411...","[{'label': 'display', 'lat': 41.50299558347314...",41.502996,-81.695876,44114,OH,The Burnham Restaurant,v-1543089334,NaN
4,"[{'id': '4bf58dd8d48988d147941735', 'name': 'D...",NaN,NaN,NaN,NaN,NaN,NaN,False,50331b85e4b012c9cf787015,NaN,...,1022,"[Cleveland, OH 44113, United States]","[{'label': 'display', 'lat': 41.48983457405474...",41.489835,-81.685635,44113,OH,Ernie's Restaurant,v-1543089334,NaN


#### Define the feature matrix including lattitude and longtitude

In [47]:
X=dataframe[["location.lat","location.lng"]]
X[0:5]

,location.lat,location.lng
0,41.499145,-81.690177
1,41.498871,-81.690149
2,41.506748,-81.686566
3,41.502996,-81.695876
4,41.489835,-81.685635


### Scale the feature matrix

In [48]:
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans 
%matplotlib inline

In [57]:
k_means = KMeans(init = "k-means++", n_clusters = 2, n_init = 12)
k_means.fit(X)


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=12, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [58]:
k_means_labels = k_means.labels_
k_means_labels

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0], dtype=int32)

In [59]:
k_means_cluster_centers = k_means.cluster_centers_
k_means_cluster_centers

array([[ 41.4483072 , -81.78716632],
       [ 41.49223834, -81.67991882]])

In [60]:

location1 = geolocator.reverse("  41.4483072 , -81.78716632")
print(location1.address)

13728, West Avenue, Jefferson, Cleveland, Cuyahoga County, Ohio, 44111, USA


In [61]:
location2 = geolocator.reverse("41.49223834, -81.67991882")
print(location2.address)

Orange Avenue, East 4th Street, Playhouse Square, Cleveland, Cuyahoga County, Ohio, 44115, USA
